In [ ]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [ ]:
documents[0]

In [ ]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

# Test connessione
print(es_client.info())

In [ ]:
# create index 'course-questions'

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

In [ ]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

In [ ]:
def elastic_search(query, course):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": course
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [ ]:
elastic_search(
    query="I just discovered the course. Can I still join?",
    course="data-engineering-zoomcamp"
)

In [ ]:
import pandas as pd

In [ ]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [ ]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [ ]:
relevance_total = []
for _q in tqdm(ground_truth):
    _doc_id = _q['document']
    _results = elastic_search(query=_q['question'], course=_q['course'])
    _relevance = [d['id'] == _doc_id for d in _results]
    relevance_total.append(_relevance)

In [ ]:
example = [
    [True, False, False, False, False], # 1, 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [False, False, False, False, False], # 0
    [False, False, False, False, False], # 0 
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1
    [True, False, False, False, False], # 1 
    [False, False, True, False, False],  # 1/3
    [False, False, False, False, False], # 0
]

# 1 => 1
# 2 => 1 / 2 = 0.5
# 3 => 1 / 3 = 0.3333
# 4 => 0.25
# 5 => 0.2
# rank => 1 / rank
# none => 0

In [ ]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [ ]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [ ]:
hit_rate(example)

In [ ]:
mrr(example)

In [ ]:
hit_rate(relevance_total), mrr(relevance_total)

In [ ]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [ ]:
def minsearch_search(query, course):
    boost = {'question': 3.0, 'section': 0.5}
    _results = index.search(query=query, filter_dict={'course': course}, boost_dict=boost, num_results=5)
    return _results

In [ ]:
relevance_total_1 = []
for _q in tqdm(ground_truth):
    _doc_id = _q['document']
    _results = minsearch_search(query=_q['question'], course=_q['course'])
    _relevance = [d['id'] == _doc_id for d in _results]
    relevance_total_1.append(_relevance)

In [ ]:
(hit_rate(relevance_total_1), mrr(relevance_total_1))

In [ ]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    for _q in tqdm(ground_truth):
        _doc_id = _q['document']
        _results = search_function(_q)
        _relevance = [d['id'] == _doc_id for d in _results]
        relevance_total.append(_relevance)
    return {'hit_rate': hit_rate(relevance_total), 'mrr': mrr(relevance_total)}

In [ ]:
evaluate(ground_truth, lambda q: elastic_search(q['question'], q['course']))

In [ ]:
evaluate(ground_truth, lambda q: minsearch_search(q['question'], q['course']))

In [ ]:
import marimo as mo

- hit-rate (recall)
- Mean Reciprocal Rank (mrr)

Compare with ES results:
```
(0.7395720769397017, 0.6032418413658963)
```

In [ ]:
mo.ui.table(df_ground_truth)